In [1]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
# prompts = pd.read_csv('/data/language-model-toxicity/prompts/prompts_with_ids.csv', index_col=0)
# pk_to_id = {tuple(pk): id_ for id_, *pk in prompts[['filename', 'begin', 'end']].itertuples()}

In [52]:
gen_files = [x for x in Path('/data/language-model-toxicity/generations/prompted').iterdir() if x.name.endswith('.jsonl')]
gen_files

[PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_affect_beta_3.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_ctrl_wiki.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt2_ft_lte2.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt3_davinci.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_ctrl.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt2.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt2_ctrl.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt1.jsonl'),
 PosixPath('/data/language-model-toxicity/generations/prompted/prompted_gens_gpt2_naughty_list.jsonl')]

In [23]:
model_name = lambda file: file.stem[len('prompted_gens_'):]
model_name(gen_files[0])

'affect_beta_3'

In [112]:
chunksize = 1000
data_len = 100_000
gen_file_iters = [pd.read_json(x, lines=True, chunksize=chunksize) for x in gen_files]
model_names = list(map(model_name, gen_files))

outs = []

offset = 0
for chunks in tqdm(zip(*gen_file_iters), total=data_len // chunksize):
    chunk_len = len(chunks[0])
    out = pd.DataFrame(data=range(offset, offset + chunk_len),
                       columns=['id'])

    out['prompt'] = [x['text'] for x in chunks[0]['prompt']]

    gens = pd.DataFrame([c['generations'] for c in chunks]).T.applymap(lambda x: [{'text': y['text'], 'toxicity': y['toxicity']} for y in x])
    gens.columns = model_names
    out['generations'] = gens.to_dict(orient='records')

    outs.append(out)

    offset += len(chunks[0])

In [118]:
df = pd.concat(outs, ignore_index=True)

In [ ]:
df.to_json('~/Desktop/generation_demo_v1.json', orient='records')